In [1]:
AGENT_NAME = 'default_PPO_citylearn_challenge_2022_phase_2_Building_6_20_bins_500'
DATASET_NAME = 'citylearn_challenge_2022_phase_2' #only action is electrical storage
SURROGATE = 'surrogates/imitator.pth'

In [2]:
from stable_baselines3 import PPO

from citylearn.data import DataSet

from art.estimators.classification import PyTorchClassifier as classifier

from KBMproject.mybb import BrendelBethgeAttack as BBA

import logging
import pandas as pd
import numpy as np
import torch
from torch.nn import CrossEntropyLoss

import KBMproject.utilities as utils

%matplotlib inline

In [3]:
schema = DataSet.get_schema(DATASET_NAME)

In [4]:
agent = PPO.load(path=f"{AGENT_NAME}")
print('Model loaded from storage')

Model loaded from storage


In [5]:
bins = agent.action_space[0].n
env = utils.make_discrete_env(schema=schema,  
                        action_bins=bins,
                        seed=42)

In [6]:
import os
path = os.path.join(os.getcwd(), '..','observation_masks.csv')
path = os.path.normpath(path) #resolve '..'
observation_masks = pd.read_csv(path)

In [9]:
kwargs = dict(norm=np.inf,
        targeted=False,
        overshoot= 2.0, #increase for surrogate
        steps=100,
        lr=1e-3,
        lr_decay=0.5,
        lr_num_decay=20,
        momentum=0.8,
        binary_search_steps=10,
        init_size=1_000_000, #default 100, finds sample matching the target class through iterative random search
        batch_size=1000, 
        )
attack = BBA(estimator=classifier(
                        model=torch.load(SURROGATE),
                        loss=CrossEntropyLoss(), 
                        nb_classes=agent.action_space[0].n, 
                        input_shape=agent.observation_space.shape,
                        device_type='gpu',
                        clip_values = (agent.observation_space.low.min(),agent.observation_space.high.max())
                        ),
            **kwargs
             )

In [10]:
logging.basicConfig(level=logging.WARN)
kpi, obs, adv_obs, asr = utils.eval_untargeted_attack(agent, env, attack,
                                                      mask=observation_masks['mask_time'].to_numpy(),
                                                      time_steps=10)

100%|██████████| 10/10 [01:05<00:00,  6.59s/it, ASR=0.3]
